In [2]:
from nltk.tokenize import word_tokenize
import nltk
import random
from nltk.corpus import movie_reviews
import pickle

short_pos = open("C:/Users/yuxia/Documents/Pyhton_Dataset/positive.txt","r").read()
short_neg = open("C:/Users/yuxia/Documents/Pyhton_Dataset/negative.txt","r").read()
documents = []
#j is a adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]
all_words = []

for p in short_pos.split('\n'):
    documents.append((p,"pos"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())
            
for p in short_neg.split('\n'):
    documents.append((p,"neg"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())           
            
save_documents = open("documents.pickle","wb")
pickle.dump(documents,save_documents)
save_documents.close()


#It's better to pickle every classifier
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:5000]
save_word_features = open("word_features5k.pickle","wb")
pickle.dump(word_features,save_word_features)
save_word_features.close()


def find_features(document):
    #The sets module provides classes for constructing and manipulating unordered collections of unique elements.
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)#(true or false)
    return features
# set(documents)
featuresets = [(find_features(rev),category) for (rev,category) in documents]
# featuresets
training_set = featuresets[:10000]
testing_set = featuresets[10000:]


In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC 
from nltk.classify import ClassifierI
from statistics import mode

classifier = nltk.NaiveBayesClassifier.train(training_set)
# print("NaiveBayesClassifier accuracy percent",(nltk.classify.accuracy(classifier,testing_set))*100)
# classifier.show_most_informative_features(15) 


save_classifier = open("OriginalNaiveBayesClassifier5k.pickle","wb")
pickle.dump(classifier,save_classifier)
save_classifier.close()


MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
# print("MNB_classifier accuracy percent:",(nltk.classify.accuracy(MNB_classifier,testing_set))*100)

save_classifier = open("MNB_Classifier5k.pickle","wb")
pickle.dump(MNB_classifier,save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
# print("BernoulliNB_classifier accuracy percent:",(nltk.classify.accuracy(BernoulliNB_classifier,testing_set))*100)

save_classifier = open("BernoulliNB_Classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier,save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
# print("LogisticRegression_classifier accuracy percent:",(nltk.classify.accuracy(LogisticRegression_classifier,testing_set))*100)

save_classifier = open("LogisticRegression_Classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier,save_classifier)
save_classifier.close()

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
# print("SGDClassifier_classifier accuracy percent:",(nltk.classify.accuracy(SGDClassifier_classifier,testing_set))*100)

save_classifier = open("SGDClassifier_Classifier5k.pickle","wb")
pickle.dump(SGDClassifier_classifier,save_classifier)
save_classifier.close()

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
# print("SVC_classifier accuracy percent:",(nltk.classify.accuracy(SVC_classifier,testing_set))*100)

save_classifier = open("SVC_Classifier5k.pickle","wb")
pickle.dump(SVC_classifier,save_classifier)
save_classifier.close()

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
# print("LinearSVC_classifier accuracy percent:",(nltk.classify.accuracy(LinearSVC_classifier,testing_set))*100)

save_classifier = open("LinearSVC_Classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier,save_classifier)
save_classifier.close()

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
# print("NuSVC_classifier accuracy percent:",(nltk.classify.accuracy(NuSVC_classifier,testing_set))*100)

save_classifier = open("NuSVC_Classifier5k.pickle","wb")
pickle.dump(NuSVC_classifier,save_classifier)
save_classifier.close()


In [7]:

def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats)


class  VoteClassifier(ClassifierI):
    #*classifiers means a list of exceeding parameters. There is no parameter required here, so every parameter is in the list.
    def __init__(self,*classifiers):
        self._classifiers = classifiers
    def classify(self,features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self,features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        choice_votes = votes.count(mode(votes))
        conf = choice_votes/len(votes)
        return conf
    
    
    
voted_classifier = VoteClassifier(classifier, MNB_classifier,LogisticRegression_classifier, SGDClassifier_classifier,SVC_classifier,LinearSVC_classifier,NuSVC_classifier)
# print("voted_classifier accuracy percent :",(nltk.classify.accuracy(voted_classifier,testing_set))*100)



voted_classifier accuracy percent : 70.18072289156626
